In [1]:
#Import libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import timeit

#Display all output not just last 
from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = "all"

#Read csv and json files and store to 'stockcards_df' and 'cat_class_df' variables respectively
stockcards_df = pd.read_csv('stockcards.csv', sep = ';')
cat_class_df = pd.read_json('cat_class.json')

#Check both variables are of DataFrame type
print("\nData structure of stockcards_df: {}".format(type(stockcards_df)))
print("Data structure of cat_class_df: {}".format(type(cat_class_df)))


Data structure of stockcards_df: <class 'pandas.core.frame.DataFrame'>
Data structure of cat_class_df: <class 'pandas.core.frame.DataFrame'>


In [2]:
#Rename 'StkISN' column to 'StockISN'
#Print first 5 rows of stockcards_df for quick visual overview
stockcards_df = stockcards_df.rename(columns = {'StkISN':'StockISN'})
print(stockcards_df.head())

        Date Type   RefNo  SNO  Cur  TUPrice   ODAmt      Amt   Worth  \
0  13/1/2011  ICG  129287    0  US$     4.00  2000.0 -2594.00 -2542.0   
1  15/4/2011  ICG  129596    0  US$     4.64  2320.0 -2906.96 -2819.0   
2  15/6/2011  ICG  129773    0  US$     4.74  2370.0 -2942.36 -2855.0   
3   9/8/2011  ICG  129929    0  US$     4.43  2215.0 -2715.59 -2636.0   
4   2/3/2012  ICG  130559    0  US$     4.32  4320.0 -5430.24 -5279.0   

  Customer Code  StockISN  
0          IT22   10002.0  
1          IT22   10002.0  
2          IT22   10002.0  
3          IT22   10002.0  
4          IT22   10002.0  


In [3]:
#Print first 5 rows of cat_class_df for quick visual overview
print(cat_class_df.head())

   StockISN CatCode
0     30114     BAM
1     10621    PPDR
2     10001      AS
3     10002     MLG
4     10003      AS


In [4]:
#Merge both dataframes together using left outer join on column 'StockISN'
stockcards = pd.merge(stockcards_df, cat_class_df, on = 'StockISN', how = 'left')
    
#Check stockcards is dataframe type after merging
print("Data structure of stockcards: {}\n".format(type(stockcards)))
print(stockcards.head())

Data structure of stockcards: <class 'pandas.core.frame.DataFrame'>

        Date Type   RefNo  SNO  Cur  TUPrice   ODAmt      Amt   Worth  \
0  13/1/2011  ICG  129287    0  US$     4.00  2000.0 -2594.00 -2542.0   
1  15/4/2011  ICG  129596    0  US$     4.64  2320.0 -2906.96 -2819.0   
2  15/6/2011  ICG  129773    0  US$     4.74  2370.0 -2942.36 -2855.0   
3   9/8/2011  ICG  129929    0  US$     4.43  2215.0 -2715.59 -2636.0   
4   2/3/2012  ICG  130559    0  US$     4.32  4320.0 -5430.24 -5279.0   

  Customer Code  StockISN CatCode  
0          IT22   10002.0     MLG  
1          IT22   10002.0     MLG  
2          IT22   10002.0     MLG  
3          IT22   10002.0     MLG  
4          IT22   10002.0     MLG  


In [5]:
#Check the dataframe shape, column names and index
stockcards.shape
stockcards.columns
stockcards.index

#Check the datatypes and NA
stockcards.info()

(56086, 12)

Index(['Date', 'Type', 'RefNo', 'SNO', 'Cur', 'TUPrice', 'ODAmt', 'Amt',
       'Worth', 'Customer Code', 'StockISN', 'CatCode'],
      dtype='object')

Int64Index([    0,     1,     2,     3,     4,     5,     6,     7,     8,
                9,
            ...
            56076, 56077, 56078, 56079, 56080, 56081, 56082, 56083, 56084,
            56085],
           dtype='int64', length=56086)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 56086 entries, 0 to 56085
Data columns (total 12 columns):
Date             56086 non-null object
Type             56086 non-null object
RefNo            56086 non-null int64
SNO              56086 non-null int64
Cur              55752 non-null object
TUPrice          55822 non-null float64
ODAmt            55822 non-null float64
Amt              55822 non-null float64
Worth            55822 non-null float64
Customer Code    55540 non-null object
StockISN         55542 non-null float64
CatCode          55542 non-null object
dtypes: float64(5), int64(2), object(5)
memory usage: 5.6+ MB


In [6]:
#remove spaces from columns
stockcards.columns = stockcards.columns.str.replace(' ', '')
print(stockcards.head())

        Date Type   RefNo  SNO  Cur  TUPrice   ODAmt      Amt   Worth  \
0  13/1/2011  ICG  129287    0  US$     4.00  2000.0 -2594.00 -2542.0   
1  15/4/2011  ICG  129596    0  US$     4.64  2320.0 -2906.96 -2819.0   
2  15/6/2011  ICG  129773    0  US$     4.74  2370.0 -2942.36 -2855.0   
3   9/8/2011  ICG  129929    0  US$     4.43  2215.0 -2715.59 -2636.0   
4   2/3/2012  ICG  130559    0  US$     4.32  4320.0 -5430.24 -5279.0   

  CustomerCode  StockISN CatCode  
0         IT22   10002.0     MLG  
1         IT22   10002.0     MLG  
2         IT22   10002.0     MLG  
3         IT22   10002.0     MLG  
4         IT22   10002.0     MLG  


In [7]:
#To add S$ when cur is null
values = {'Cur':'S$'}
stockcards.fillna(value = values, inplace = True)

#To add “CASH” when Customer Code is null
values = {'CustomerCode':'CASH'}
stockcards.fillna(value = values, inplace = True)

#Replace SIN with S$
stockcards.replace(to_replace = ["SIN", "SGD"], value = "S$", inplace = True)
#Replace US, US$, USD with USD$
stockcards.replace(to_replace = ['US','US$','USD'], value = 'USD$', inplace = True)

#Change all customer code to capitalised
stockcards["CustomerCode"] = stockcards["CustomerCode"].str.upper()
#Add a new column "Customer Region" with initial letter for customer code
stockcards["CustomerRegion"] = stockcards["CustomerCode"].str[:1]
#Replace customer Region “R” to “N”
#Replace customer Region “C” to “N”
stockcards["CustomerRegion"].replace(to_replace = ["R", "C"], value = "N", inplace = True)

#To count total number of null values
print(stockcards.isnull().sum())   
#To drop row if there is null value in cell. 55539 rows left 
stockcards.dropna(inplace = True) 
#Recount total number of null values
print(stockcards.isnull().sum())
#To count the number of rows and colm
print(stockcards.shape)

Date                0
Type                0
RefNo               0
SNO                 0
Cur                 0
TUPrice           264
ODAmt             264
Amt               264
Worth             264
CustomerCode        0
StockISN          544
CatCode           544
CustomerRegion      0
dtype: int64
Date              0
Type              0
RefNo             0
SNO               0
Cur               0
TUPrice           0
ODAmt             0
Amt               0
Worth             0
CustomerCode      0
StockISN          0
CatCode           0
CustomerRegion    0
dtype: int64
(55542, 13)


In [8]:
#Extract year, month, day from date column
stockcards['Date'] = pd.to_datetime(stockcards['Date'])
stockcards['Year'] = pd.DatetimeIndex(stockcards['Date']).year
stockcards['Month'] = pd.DatetimeIndex(stockcards['Date']).month
stockcards['Day'] = pd.DatetimeIndex(stockcards['Date']).day
print(stockcards.head())

        Date Type   RefNo  SNO   Cur  TUPrice   ODAmt      Amt   Worth  \
0 2011-01-13  ICG  129287    0  USD$     4.00  2000.0 -2594.00 -2542.0   
1 2011-04-15  ICG  129596    0  USD$     4.64  2320.0 -2906.96 -2819.0   
2 2011-06-15  ICG  129773    0  USD$     4.74  2370.0 -2942.36 -2855.0   
3 2011-09-08  ICG  129929    0  USD$     4.43  2215.0 -2715.59 -2636.0   
4 2012-02-03  ICG  130559    0  USD$     4.32  4320.0 -5430.24 -5279.0   

  CustomerCode  StockISN CatCode CustomerRegion  Year  Month  Day  
0         IT22   10002.0     MLG              I  2011      1   13  
1         IT22   10002.0     MLG              I  2011      4   15  
2         IT22   10002.0     MLG              I  2011      6   15  
3         IT22   10002.0     MLG              I  2011      9    8  
4         IT22   10002.0     MLG              I  2012      2    3  


In [9]:
#Change TUPrice, Amt and ODAmt to Positive Values. Assume any discrepancies in value as input error since only 5 negative ODAmt value.
stockcards["TUPrice"] = stockcards["TUPrice"].abs()
stockcards["Amt"] = stockcards["Amt"].abs()
stockcards["ODAmt"] = stockcards["ODAmt"].abs()

In [10]:
#Create a quantity and profit column to analyze stock takes, when to restock
stockcards['Quantity'] = stockcards['ODAmt'] / stockcards['TUPrice']
stockcards['Profit'] = abs(stockcards['Amt']) - abs(stockcards['Worth'])

In [11]:
#Discovered some values have trailing white spaces while performing uniqueness check
#Create a function to trim whitespace from both ends of each value across all string series in dataframe
#Applymap works element-wise on a DataFrame
def trim_all_columns(stockcards):
    trim_strings = lambda x: x.strip() if isinstance(x, str) else x
    return stockcards.applymap(trim_strings)

#Trimming whitespace from data (Apply to stockcards dataframe)
stockcards = trim_all_columns(stockcards)
stockcards

,Date,Type,RefNo,SNO,Cur,TUPrice,ODAmt,Amt,Worth,CustomerCode,StockISN,CatCode,CustomerRegion,Year,Month,Day,Quantity,Profit
0,2011-01-13,ICG,129287,0,USD$,4.00,2000.0,2594.00,-2542.00,IT22,10002.0,MLG,I,2011,1,13,500.0,52.00
1,2011-04-15,ICG,129596,0,USD$,4.64,2320.0,2906.96,-2819.00,IT22,10002.0,MLG,I,2011,4,15,500.0,87.96
2,2011-06-15,ICG,129773,0,USD$,4.74,2370.0,2942.36,-2855.00,IT22,10002.0,MLG,I,2011,6,15,500.0,87.36
3,2011-09-08,ICG,129929,0,USD$,4.43,2215.0,2715.59,-2636.00,IT22,10002.0,MLG,I,2011,9,8,500.0,79.59
4,2012-02-03,ICG,130559,0,USD$,4.32,4320.0,5430.24,-5279.00,IT22,10002.0,MLG,I,2012,2,3,1000.0,151.24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56081,2019-04-27,ICG,137292,2,S$,6.20,1550.0,1550.00,-1175.91,IG01,43512.0,ML,I,2019,4,27,250.0,374.09
56082,2019-04-27,ICG,137292,3,S$,4.80,374.4,374.40,-296.03,IG01,10097.0,KPR,I,2019,4,27,78.0,78.37
56083,2019-04-27,ICG,137292,4,S$,4.80,412.8,412.80,-307.67,IG01,41440.0,KPR,I,2019,4,27,86.0,105.13
56084,2019-04-27,ICG,137293,1,S$,60.00,3000.0,3000.00,-2500.00,IA14,46382.0,AAN,I,2019,4,27,50.0,500.00


In [12]:
#Class to store different column checking methods (e.g. check category, check uniqueness)
class CheckColumns:    
    #Method to check number of unique values in each column
    #Check suitability to be converted to categorical type
    def unique_values():
        print("Currently there are {} columns.\n".format(len(stockcards.columns)))
        for col in stockcards:
            print("{} has {} unique values.".format(col, len(stockcards[col].unique())))
        print("\n")
    #Method to check number of types for each category column
    #Ensure cleaning was performed properly (e.g. 'Type' column only has ICG/IGX)
    def unique_category():
        for col in stockcards.select_dtypes('category'):
            print("Column: {}".format(col))
            print(stockcards[col].unique())
            print("\n")

#Change Type, Cur, Customer Code, CatCode, CustomerRegion to category type for meaningful analysis
stockcards['Type'] = stockcards['Type'].astype('category')
stockcards['Cur'] = stockcards['Cur'].astype('category')
stockcards['CustomerCode'] = stockcards['CustomerCode'].astype('category')
stockcards['CatCode'] = stockcards['CatCode'].astype('category')
stockcards['CustomerRegion'] = stockcards['CustomerRegion'].astype('category')

#Execute uniqueness check for each column        
CheckColumns.unique_values()
#Execute category check
CheckColumns.unique_category()

Currently there are 18 columns.

Date has 2313 unique values.
Type has 2 unique values.
RefNo has 10011 unique values.
SNO has 66 unique values.
Cur has 3 unique values.
TUPrice has 2004 unique values.
ODAmt has 20928 unique values.
Amt has 21550 unique values.
Worth has 27794 unique values.
CustomerCode has 179 unique values.
StockISN has 4129 unique values.
CatCode has 179 unique values.
CustomerRegion has 6 unique values.
Year has 9 unique values.
Month has 12 unique values.
Day has 31 unique values.
Quantity has 12598 unique values.
Profit has 32391 unique values.


Column: Type
[ICG, ICX]
Categories (2, object): [ICG, ICX]


Column: Cur
[USD$, S$, M$]
Categories (3, object): [USD$, S$, M$]


Column: CustomerCode
[IT22, IP06, IT12, IZ01, IA09, ..., SG04, IS01, SG08, IA01, SH22]
Length: 179
Categories (179, object): [IT22, IP06, IT12, IZ01, ..., IS01, SG08, IA01, SH22]


Column: CatCode
[MLG, AS, BW, BN, M018, ..., BOLT, M032, BAM, AC, SSBN]
Length: 179
Categories (179, object): [ML

In [13]:
#For ODAmt = 0, change to Amt value. This is to ensure currency translation consistency.
#numpy generally performs better than pandas for 50K rows or less. pandas generally performs better than numpy for 500K rows or more
#For this dataset, np.where takes much less time compared to pd.replace (160,000 times faster)
# %time stockcards['ODAmt'] = np.where(stockcards['ODAmt'] == 0, stockcards['Amt'], stockcards['ODAmt']) #997 µs   
# %time stockcards.ODAmt.replace(0, stockcards.Amt, inplace = True) #2min 38s
stockcards['ODAmt'] = np.where(stockcards['ODAmt'] == 0, stockcards['Amt'], stockcards['ODAmt'])   

In [14]:
#Check there is no row with ODAmt = 0 but Amt > 0
stockcards.loc[(stockcards.ODAmt == 0) & (stockcards.Amt > 0)]

,Date,Type,RefNo,SNO,Cur,TUPrice,ODAmt,Amt,Worth,CustomerCode,StockISN,CatCode,CustomerRegion,Year,Month,Day,Quantity,Profit


In [15]:
#Check for rows with TUPrice, ODAmt, Amt, Worth all equal to zero
#They provide little to no analytics value
stockcards.loc[(stockcards.TUPrice == 0) & (stockcards.ODAmt == 0) & (stockcards.Amt == 0) & (stockcards.Worth == 0)]

,Date,Type,RefNo,SNO,Cur,TUPrice,ODAmt,Amt,Worth,CustomerCode,StockISN,CatCode,CustomerRegion,Year,Month,Day,Quantity,Profit
9,2013-11-29,ICG,132297,0,USD$,0.0,0.0,0.0,0.0,IP06,10002.0,MLG,I,2013,11,29,NaN,0.0
652,2013-11-29,ICG,132297,0,USD$,0.0,0.0,0.0,0.0,IP06,10043.0,MLG,I,2013,11,29,NaN,0.0
4213,2013-02-16,ICG,131481,0,S$,0.0,0.0,0.0,0.0,IJ01,30075.0,DSK,I,2013,2,16,NaN,0.0
4767,2012-07-27,ICX,921275,0,S$,0.0,0.0,0.0,0.0,CASH,30262.0,PR04,N,2012,7,27,NaN,0.0
6753,2013-09-26,ICG,132096,0,S$,0.0,0.0,0.0,0.0,IT26,40175.0,MH,I,2013,9,26,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54745,2019-01-29,ICG,137091,7,S$,0.0,0.0,0.0,0.0,IK11,10077.0,F,I,2019,1,29,NaN,0.0
55281,2019-05-03,ICG,137167,1,S$,0.0,0.0,0.0,0.0,IA14,40412.0,WP,I,2019,5,3,NaN,0.0
55282,2019-05-03,ICG,137167,2,S$,0.0,0.0,0.0,0.0,IA14,43516.0,NEE,I,2019,5,3,NaN,0.0
55283,2019-05-03,ICG,137167,3,S$,0.0,0.0,0.0,0.0,IA14,43515.0,NEE,I,2019,5,3,NaN,0.0


In [16]:
#Delete all rows with TUPrice, ODAmt, Amt, Worth all equal to zero. No analytics value.
stockcards.drop(stockcards.loc[(stockcards.TUPrice == 0) & (stockcards.ODAmt == 0) & (stockcards.Amt == 0) & (stockcards.Worth == 0)].index, inplace = True)

In [17]:
#Double check these rows are removed
stockcards.loc[(stockcards.TUPrice == 0) & (stockcards.ODAmt == 0) & (stockcards.Amt == 0) & (stockcards.Worth == 0)]

,Date,Type,RefNo,SNO,Cur,TUPrice,ODAmt,Amt,Worth,CustomerCode,StockISN,CatCode,CustomerRegion,Year,Month,Day,Quantity,Profit


In [18]:
stockcards

,Date,Type,RefNo,SNO,Cur,TUPrice,ODAmt,Amt,Worth,CustomerCode,StockISN,CatCode,CustomerRegion,Year,Month,Day,Quantity,Profit
0,2011-01-13,ICG,129287,0,USD$,4.00,2000.0,2594.00,-2542.00,IT22,10002.0,MLG,I,2011,1,13,500.0,52.00
1,2011-04-15,ICG,129596,0,USD$,4.64,2320.0,2906.96,-2819.00,IT22,10002.0,MLG,I,2011,4,15,500.0,87.96
2,2011-06-15,ICG,129773,0,USD$,4.74,2370.0,2942.36,-2855.00,IT22,10002.0,MLG,I,2011,6,15,500.0,87.36
3,2011-09-08,ICG,129929,0,USD$,4.43,2215.0,2715.59,-2636.00,IT22,10002.0,MLG,I,2011,9,8,500.0,79.59
4,2012-02-03,ICG,130559,0,USD$,4.32,4320.0,5430.24,-5279.00,IT22,10002.0,MLG,I,2012,2,3,1000.0,151.24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56081,2019-04-27,ICG,137292,2,S$,6.20,1550.0,1550.00,-1175.91,IG01,43512.0,ML,I,2019,4,27,250.0,374.09
56082,2019-04-27,ICG,137292,3,S$,4.80,374.4,374.40,-296.03,IG01,10097.0,KPR,I,2019,4,27,78.0,78.37
56083,2019-04-27,ICG,137292,4,S$,4.80,412.8,412.80,-307.67,IG01,41440.0,KPR,I,2019,4,27,86.0,105.13
56084,2019-04-27,ICG,137293,1,S$,60.00,3000.0,3000.00,-2500.00,IA14,46382.0,AAN,I,2019,4,27,50.0,500.00


In [19]:
#Save cleaned dataframe as csv file
stockcards.to_csv('C:\\Users\\user\\Desktop\\Year 2 NBS\\Semester 1\\BC0401 PROGRAMMING FOR BUSINESS ANALYTICS\\Project\\Clean_stockcards.csv', index = False)